### Imports e Instância do spark

In [38]:
import matplotlib as plt
import pandas as pd
from datetime import datetime


No module named pandas_profiling
Traceback (most recent call last):
ImportError: No module named pandas_profiling



In [15]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


### Exploração dos dados

### Consulta 1

Neste ponto deve-se reproduzir o que esta consulta faz
-- Quantidade de reprovações de cada aluno:

```
 select id_user, count(id_user) as reprovacoes 
   from situacao_matricula_diario 
       where aprovacao='Reprovado' 
         group by id_user 
         having reprovacoes>0 
         order by reprovacoes desc;
```

In [5]:
df_situacao_matricula_diario = spark.read.csv(
    "wasbs://dadoscru@storhack.blob.core.windows.net/situacao_matricula_diario.csv",
    header=True,
    sep=",",
)

df_situacao_matricula_diario.createTempView("situacao_matricula_diario")

In [31]:
df_situacao_matricula_diario = spark.sql("""
    select 
        id_user, 
        count(id_user) as reprovacoes 
    from 
        situacao_matricula_diario 
    where aprovacao='Reprovado' 
    group by id_user 
    having reprovacoes>0 
    order by reprovacoes desc
"""
)

In [ ]:
df_situacao_matricula_diario.show(5, -1)

In [32]:
df = df.select("*").toPandas()

In [36]:
df.head()

  id_user  reprovacoes
0   13360           10
1   13829           10
2   14366            9
3    7716            9
4    2762            8

In [37]:
pandas_profiling.ProfileReport(df)

name 'pandas_profiling' is not defined
Traceback (most recent call last):
NameError: name 'pandas_profiling' is not defined



### Consulta 2

-- Quantidade de postagens dos alunos no forum nos últimos 30 dias:

```select u.id_user, sum(f.id_forum is not null) postagens 
     from usuario u 
     LEFT OUTER JOIN (select id_user, id_forum 
                        from forum 
                       where data_post>strftime('%s','now') - 606024*30) AS f 
     ON u.id_user = f.id_user 
    group by u.id_user 
    order by postagens asc;
```

In [39]:
df_usuarios = spark.read.csv(
    "wasbs://dadoscru@storhack.blob.core.windows.net/usuarios.csv",
    header=True,
    sep=",",
)

df_usuarios.createTempView("usuarios")

df_forum = spark.read.csv(
    "wasbs://dadoscru@storhack.blob.core.windows.net/AVA.forum.csv",
    header=True,
    sep=",",
)

df_forum.createTempView("forum")

u"Temporary view 'usuarios' already exists;"
Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/dataframe.py", line 163, in createTempView
    self._jdf.createTempView(name)
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 71, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u"Temporary view 'usuarios' already exists;"



In [40]:
df_usuarios.printSchema()

root
 |-- id_user: string (nullable = true)
 |-- id_login: string (nullable = true)
 |-- data_criacao: string (nullable = true)
 |-- removido: string (nullable = true)
 |-- suspenso: string (nullable = true)
 |-- cidade: string (nullable = true)

In [41]:
df_forum.printSchema()

root
 |-- id_forum: integer (nullable = true)
 |-- data_limite: integer (nullable = true)
 |-- id_user: integer (nullable = true)
 |-- id_course: integer (nullable = true)
 |-- id_discusao: integer (nullable = true)
 |-- inicio_discusao: integer (nullable = true)
 |-- termino_discusao: integer (nullable = true)
 |-- data_post: integer (nullable = true)
 |-- tamanho_mensagem: integer (nullable = true)
 |-- anexos: integer (nullable = true)
 |-- current_timestamp: long (nullable = true)

In [57]:
df_forum.select(
    'data_limite',
    to_timestamp(from_unixtime('data_limite')).alias('data_limite_edited'),
    to_timestamp(from_unixtime('data_limite')).alias('data_limite_edited_2').cast(DateType()),
    date_sub(to_timestamp(from_unixtime('data_limite')), -30).alias('data_limite_edited_3'),
    
    'data_post',
    to_timestamp(from_unixtime('data_post')).alias('data_post_edited'),
    to_timestamp(from_unixtime('data_post')).alias('data_post_edited_2').cast(DateType()),
).show(5, -1)
# ).printSchema()

+-----------+-------------------+--------------------+--------------------+----------+-------------------+------------------+
|data_limite|data_limite_edited |data_limite_edited_2|data_limite_edited_3|data_post |data_post_edited   |data_post_edited_2|
+-----------+-------------------+--------------------+--------------------+----------+-------------------+------------------+
|1588813200 |2020-05-07 01:00:00|2020-05-07          |2020-06-06          |1580240405|2020-01-28 19:40:05|2020-01-28        |
|0          |1970-01-01 00:00:00|1970-01-01          |1970-01-31          |1580240405|2020-01-28 19:40:05|2020-01-28        |
|0          |1970-01-01 00:00:00|1970-01-01          |1970-01-31          |1586215240|2020-04-06 23:20:40|2020-04-06        |
|0          |1970-01-01 00:00:00|1970-01-01          |1970-01-31          |1588625946|2020-05-04 20:59:06|2020-05-04        |
|0          |1970-01-01 00:00:00|1970-01-01          |1970-01-31          |1588522550|2020-05-03 16:15:50|2020-05-03  

In [ ]:
select u.id_user, sum(f.id_forum is not null) postagens 
     from usuario u 
     LEFT OUTER JOIN (select id_user, id_forum 
                        from forum 
                       where data_post>strftime('%s','now') - 606024*30) AS f 
     ON u.id_user = f.id_user 
    group by u.id_user 
    order by postagens asc;

In [59]:
df_usuarios.alias(
    "U"
).join(
    other=df_forum.alias("F").withColumn(
        "data_post",
        to_timestamp(from_unixtime('data_post'))
    ).withColumn(
        "data_limite",
        to_timestamp(from_unixtime('data_limite'))
    ).withColumn(
        "id_forum",
        col("id_forum").cast(LongType())
    ).filter(
        date_sub(to_timestamp(from_unixtime('data_post')), -30)
    ).select(
        "id_forum",
        "id_user",
    ),
    on=[
        col("U.id_user") == col("F.id_user")
    ],
    how="left"
).groupBy(
    col("U.id_user"),
).agg(
    count("id_forum").alias("postagens")
).sort(
    col("id_user")
).show(5,-1)

u"cannot resolve 'from_unixtime(`data_post`, 'yyyy-MM-dd HH:mm:ss')' due to data type mismatch: argument 1 requires bigint type, however, '`data_post`' is of timestamp type.;;\n'Filter date_sub(to_timestamp(from_unixtime('data_post, yyyy-MM-dd HH:mm:ss, None), None), -30)\n+- Project [cast(id_forum#97 as bigint) AS id_forum#1074L, data_limite#1062, id_user#99, id_course#100, id_discusao#101, inicio_discusao#102, termino_discusao#103, data_post#1050, tamanho_mensagem#105, anexos#106, current_timestamp#184L]\n   +- Project [id_forum#97, to_timestamp(from_unixtime('data_limite, yyyy-MM-dd HH:mm:ss, None), None) AS data_limite#1062, id_user#99, id_course#100, id_discusao#101, inicio_discusao#102, termino_discusao#103, data_post#1050, tamanho_mensagem#105, anexos#106, current_timestamp#184L]\n      +- Project [id_forum#97, data_limite#98, id_user#99, id_course#100, id_discusao#101, inicio_discusao#102, termino_discusao#103, to_timestamp(from_unixtime('data_post, yyyy-MM-dd HH:mm:ss, None), 

In [46]:
df_forum.filter(col("id_user") == "35844").show()

+--------+-----------+-------+---------+-----------+---------------+----------------+---------+----------------+------+-----------------+
|id_forum|data_limite|id_user|id_course|id_discusao|inicio_discusao|termino_discusao|data_post|tamanho_mensagem|anexos|current_timestamp|
+--------+-----------+-------+---------+-----------+---------------+----------------+---------+----------------+------+-----------------+
+--------+-----------+-------+---------+-----------+---------------+----------------+---------+----------------+------+-----------------+

In [25]:
spark.read.table("forum").show()

+--------+-----------+-------+---------+-----------+---------------+----------------+----------+----------------+------+-----------------+
|id_forum|data_limite|id_user|id_course|id_discusao|inicio_discusao|termino_discusao| data_post|tamanho_mensagem|anexos|current_timestamp|
+--------+-----------+-------+---------+-----------+---------------+----------------+----------+----------------+------+-----------------+
|    5190| 1588813200|   8813|     2703|       8138|              0|               0|1580240405|             112|  null|       1606359004|
|    5189|          0|  29010|     2703|       8137|              0|               0|1580240405|             180|  null|       1606359004|
|    5189|          0|  18950|     2703|       8137|              0|               0|1586215240|            1481|  null|       1606359004|
|    5189|          0|  18952|     2703|       8137|              0|               0|1588625946|            1405|  null|       1606359004|
|    5189|          0|  189

In [ ]:
select u.id_user, sum(f.id_forum is not null) postagens 
     from usuario u 
     LEFT OUTER JOIN (select id_user, id_forum 
                        from forum 
                       where data_post>strftime('%s','now') - 606024*30) AS f 
     ON u.id_user = f.id_user 
    group by u.id_user 
    order by postagens asc;

In [ ]:
spark.sql("""
    select 
        u.id_user, 
        sum(f.id_forum) postagens 
    from usuario u 
    LEFT OUTER JOIN (
                    select 
                        id_user, 
                        id_forum 
                    from forum 
                    where data_post > strftime('%s','now') - 606024*30) AS f 
    ON u.id_user = f.id_user 
    group by u.id_user 
    order by postagens asc
""").show(10, -1)

### Consulta 3

-- Quantidade de acessos ao AVA nos últimos 30 dias:

`
select u.id_user, sum(l.id is not null) acessos from usuario u LEFT OUTER JOIN (select userid, id from logs where timecreated>strftime('%s','now') - 606024*30) AS l ON u.id_user = l.userid group by u.id_user order by acessos asc;`

In [ ]:
# Dataframe escolaridade
dfescolaridade = spark.read.csv('wasbs://dadoscru@storhack.blob.core.windows.net/sgn.escolaridade.csv', header=True, inferSchema=True)

In [ ]:
dfescolaridade.select("genero").groupBy("genero").count().show()

+------+-----+
|genero|count|
+------+-----+
|     F| 2048|
|     M| 8310|
+------+-----+

In [ ]:
dfescolaridade.select("tipo_escola").groupBy("tipo_escola").count().show()

+-----------------+-----+
|      tipo_escola|count|
+-----------------+-----+
|  Publica Federal|  278|
| Pública Estadual| 6197|
|       Particular| 2773|
|Pública Municipal|  786|
|    Não declarada|  324|
+-----------------+-----+

In [ ]:
# Dataframe Turmas
dfturmas = spark.read.csv('wasbs://dadoscru@storhack.blob.core.windows.net/sgn.turma.csv', header=True, inferSchema=True)
dfturmas = dfturmas.drop('requisito_obrigatorio')

In [ ]:
### Falta aqui filtrar para retirar os casos onde a coluna cancelada = 'FALSO', pois este cursos foram cancelados
dfturmas.select("titulo", "ano").groupBy("ano","titulo").count().show()

+------+--------------------+-----+
|   ano|              titulo|count|
+------+--------------------+-----+
|2018/1|Técnico em Edific...|    5|
|2020/1|Técnico em Mecatr...|    5|
|2020/1| Técnico em Cerâmica|    1|
|2020/2|Técnico em Segura...|    6|
|2018/2|Técnico em Automa...|    5|
|2019/2|Técnico em Manute...|    1|
|2020/2|Técnico em Eletro...|    1|
|2019/2|Técnico em Eletro...|    1|
|2020/2|Técnico em Manute...|    1|
|2020/2|Técnico em Automa...|    3|
|2019/2| Técnico em Plástico|    1|
|2019/1|Técnico em Eletro...|    5|
|2020/1|Técnico em Edific...|    2|
|2018/2|Técnico em Edific...|    1|
|2019/2|Técnico em Desenv...|    2|
|2019/2|Técnico em Admini...|    1|
|2020/1| Técnico em Mecânica|   12|
|2019/1|Técnico em Automa...|    2|
|2020/1|Técnico em Eletro...|   19|
|2020/1|   Técnico em Têxtil|    1|
+------+--------------------+-----+
only showing top 20 rows

In [ ]:
# Create a dataframe and table from sample data
dfcursos = spark.read.csv('wasbs://dadoscru@storhack.blob.core.windows.net/cursos.csv', header=True, inferSchema=True)
dfcursos.show(10)

+---------+--------------------+---------+------------+----------+-------+--------+
|id_course|           shortname|id_diario|inicio_curso|   termino|visivel|id_turma|
+---------+--------------------+---------+------------+----------+-------+--------+
|     2169|Comunicação Oral ...|   AGR_90|  1552878000|1555038000|      1|    null|
|     3094|Otimização de Pro...|   219273|  1583290800|1583298000|      1|   45912|
|     3023|Manutenção de Máq...|   228499|  1580698800|1597460400|      1|   47828|
|     2864|Implementação de ...|   228498|  1580698800|1592190000|      1|   47828|
|     3202|Comunicação e Inf...|   261308|  1591329600|         0|      1|   54809|
|     3029|Processos Básicos...|   251610|  1583722800|1595818800|      1|   53251|
|     2690|Implementação de ...|  AGR_271|  1581303600|1588215600|      1|    null|
|     3038|Ciências Aplicada...|   251666|  1583118000|1594350000|      1|   53255|
|     3241|Comunicação e Inf...|   252289|  1593658800|         0|      1|  

In [ ]:
dfusuarios = spark.read.csv('wasbs://dadoscru@storhack.blob.core.windows.net/usuarios.csv', header=True, inferSchema=True)
dfusuarios.show(10)

+-------+--------+------------+--------+--------+------------------+
|id_user|id_login|data_criacao|removido|suspenso|            cidade|
+-------+--------+------------+--------+--------+------------------+
|  11887|  219883|  1520442468|       0|       0|    Jaraguá do Sul|
|  20440|  122116|  1538512143|       0|       0|         Canoinhas|
|  29957|  592879|  1565894867|       0|       0|Balneário Camboriú|
|  39684|  564376|  1590443653|       0|       0|         Joinville|
|  28718|  559978|  1556538396|       0|       0|Balneário Camboriú|
|  33349|  595126|  1583888041|       0|       0|           Brusque|
|  32662|  640660|  1583531468|       0|       0|         Concórdia|
|  31095|  559373|  1572753835|       0|       0|           Chapecó|
|  12281|  523484|  1521564542|       0|       0|          Blumenau|
|  38443|  652149|  1587729450|       0|       0|              Ouro|
+-------+--------+------------+--------+--------+------------------+
only showing top 10 rows

In [ ]:
from pyspark.sql.functions import col, from_unixtime, unix_timestamp
from pyspark.sql.types import DoubleType, StringType, IntegerType, LongType, TimestampType, StructType, StructField

In [ ]:
dfusuarios=dfusuarios.withColumn('data_criacao_', from_unixtime(col('data_criacao'),'yyyy-MM-dd HH:mm:ss')).withColumn('data_criacao_',col('data_criacao_').cast(TimestampType()))

In [ ]:
dfusuarios.printSchema()

root
 |-- id_user: integer (nullable = true)
 |-- id_login: integer (nullable = true)
 |-- data_criacao: integer (nullable = true)
 |-- removido: integer (nullable = true)
 |-- suspenso: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- data_criacao_: timestamp (nullable = true)

In [ ]:
dfcursos=dfcursos.withColumn('inicio_curso_', from_unixtime(col('inicio_curso'),'yyyy-MM-dd HH:mm:ss')).withColumn('inicio_curso_',col('inicio_curso_').cast(TimestampType()))

In [ ]:
dfcursos.show(10)

+---------+--------------------+---------+------------+----------+-------+--------+-------------------+
|id_course|           shortname|id_diario|inicio_curso|   termino|visivel|id_turma|      inicio_curso_|
+---------+--------------------+---------+------------+----------+-------+--------+-------------------+
|     2169|Comunicação Oral ...|   AGR_90|  1552878000|1555038000|      1|    null|2019-03-18 03:00:00|
|     3094|Otimização de Pro...|   219273|  1583290800|1583298000|      1|   45912|2020-03-04 03:00:00|
|     3023|Manutenção de Máq...|   228499|  1580698800|1597460400|      1|   47828|2020-02-03 03:00:00|
|     2864|Implementação de ...|   228498|  1580698800|1592190000|      1|   47828|2020-02-03 03:00:00|
|     3202|Comunicação e Inf...|   261308|  1591329600|         0|      1|   54809|2020-06-05 04:00:00|
|     3029|Processos Básicos...|   251610|  1583722800|1595818800|      1|   53251|2020-03-09 03:00:00|
|     2690|Implementação de ...|  AGR_271|  1581303600|158821560

In [ ]:
dfcursos=dfcursos.withColumn('termino_', from_unixtime(col('termino'),'yyyy-MM-dd HH:mm:ss')).withColumn('termino_',col('termino_').cast(TimestampType()))

In [ ]:
dfcursos.printSchema()

root
 |-- id_course: integer (nullable = true)
 |-- shortname: string (nullable = true)
 |-- id_diario: string (nullable = true)
 |-- inicio_curso: integer (nullable = true)
 |-- termino: integer (nullable = true)
 |-- visivel: integer (nullable = true)
 |-- id_turma: integer (nullable = true)
 |-- inicio_curso_: timestamp (nullable = true)
 |-- termino_: timestamp (nullable = true)

In [ ]:
dfcursos.show(10)

+---------+--------------------+---------+------------+----------+-------+--------+-------------------+-------------------+
|id_course|           shortname|id_diario|inicio_curso|   termino|visivel|id_turma|      inicio_curso_|           termino_|
+---------+--------------------+---------+------------+----------+-------+--------+-------------------+-------------------+
|     2169|Comunicação Oral ...|   AGR_90|  1552878000|1555038000|      1|    null|2019-03-18 03:00:00|2019-04-12 03:00:00|
|     3094|Otimização de Pro...|   219273|  1583290800|1583298000|      1|   45912|2020-03-04 03:00:00|2020-03-04 05:00:00|
|     3023|Manutenção de Máq...|   228499|  1580698800|1597460400|      1|   47828|2020-02-03 03:00:00|2020-08-15 03:00:00|
|     2864|Implementação de ...|   228498|  1580698800|1592190000|      1|   47828|2020-02-03 03:00:00|2020-06-15 03:00:00|
|     3202|Comunicação e Inf...|   261308|  1591329600|         0|      1|   54809|2020-06-05 04:00:00|1970-01-01 00:00:00|
|     30